In [1]:
#%run input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

Welcome to JupyROOT 6.30/06


In [2]:
iOption0 = 0

In [3]:
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
central_bins = [0,95,0,20,20,40,40,60,60,100]
N_centr = int(len(central_bins)/2)
pt_boarders = [0.2,0.3,0.4,0.5,0.6,0.7,0.9,1.1,1.4,1.9,2.5,3.5,5.0]
N_pt = len(pt_boarders)-1
file_path="/home/yoren/bnl/PHENIX/ee/AuAu/"
file_name="my-10M_phi_embed_helios_v1.root"
part_name = ["e^{+}","e^{-}"]

In [4]:
root.gSystem.Load("../AnaTrain/Run14AuAuLeptonComby/MyEvent_C.so")

0

In [5]:
root.gROOT.ProcessLine(
"TFile *input = new TFile(\"/home/yoren/bnl/PHENIX/ee/AuAu/my-10M_phi_embed_helios_v1.root\", \"READ\");\
TTree *T = (TTree *)input->Get(\"tree\");\
MyDileptonAnalysis::MyEvent *myevent = 0;\
TBranch *br = T->GetBranch(\"MyEvent\");\
br->SetAddress(&myevent);\
"
); 
from ROOT import myevent, br

In [6]:
i = 0
N_real_ev = br.GetEntries()
print(N_real_ev)
for ievent in range(N_real_ev):
    i+=1
    if i>10: break
    myevent.ClearEvent()
    br.GetEntry(ievent)
    print(myevent.GetPreciseX(),myevent.GetNtrack()) 
    if myevent.GetNtrack(): print("pt:", myevent.GetEntry(0).GetPtPrime())

9960000
0.33104899525642395 0
0.328918993473053 0
0.3377250134944916 0
0.3359299898147583 0
0.36426401138305664 0
0.3182789981365204 1
pt: 1.7531354427337646
0.33480900526046753 0
0.32921698689460754 0
0.3495129942893982 0
0.3419190049171448 0


In [7]:
infile = root.TFile.Open(file_path+file_name, "read")
tree_read = infile.Get("tree")
tree_read.SetDirectory(root.nullptr)

infile.Close()

In [8]:
branch = tree_read.GetBranch("MyEvent")
branch.SetAddress(myevent)

In [9]:
root.gROOT.ProcessLine(
"auto branch = (TBranch)*gROOT->FindObject(\"branch\");\
 branch->SetAddress(&ev);\
"
); 

input_line_105:2:16: error: no matching conversion for C-style cast from 'TObject' to 'TBranch'
 auto branch = (TBranch)*gROOT->FindObject("branch"); branch->SetAddress(&ev);
               ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/home/yoren/bnl/ROOT/install/include/TBranch.h:189:4: note: candidate constructor not viable: no known conversion from 'TObject' to 'const TBranch' for 1st argument
   TBranch(const TBranch&) = delete;             // not implemented
   ^
/home/yoren/bnl/ROOT/install/include/TBranch.h:193:4: note: candidate constructor not viable: requires 0 arguments, but 1 was provided
   TBranch();
   ^
/home/yoren/bnl/ROOT/install/include/TBranch.h:194:4: note: candidate constructor not viable: requires at least 4 arguments, but 1 was provided
   TBranch(TTree *tree, const char *name, void *address, const char *leaflist, Int_t basketsize=32000, Int_t compress = ROOT::RCompressionSetting::EAlgorithm::kInherit);
   ^
/home/yoren/bnl/ROOT/install/include/TBranch.h:195:4: note: c

In [10]:
i = 0
N_real_ev = branch.GetEntries()
print(N_real_ev)
for ievent in range(N_real_ev):
    i+=1
    if i>10: break
    myevent.ClearEvent()
    branch.GetEntry(ievent)
    print(myevent.GetPreciseX()) 

9960000
-999.0
-999.0
-999.0
-999.0
-999.0
-999.0
-999.0
-999.0
-999.0
-999.0


In [11]:
root.gROOT.ProcessLine(
"struct Traj {\
  Int_t           validhit;\
  Float_t         missing;\
  Float_t         lx;\
  Float_t         ly;\
  Float_t         lz;\
  Float_t         glx;\
  Float_t         gly;\
  Float_t         glz;\
  Int_t           clust_near;\
  Float_t         hit_near;\
  Float_t         pass_effcuts;\
  Float_t         alpha;\
  Float_t         beta;\
  Float_t         norm_charge;\
  Float_t         d_tr;\
  Float_t         dx_tr;\
  Float_t         dy_tr;\
  Float_t         d_cl;\
  Float_t         dx_cl;\
  Float_t         dy_cl;\
  Float_t         dx_hit;\
  Float_t         dy_hit;\
  Int_t           onedge;\
  Float_t         lx_err;\
  Float_t         ly_err;\
  };"
); 

In [12]:
#root.gROOT.ProcessLine(
#"include \"../AnaTrain/Run14AuAuLeptonComby/MyEvent.h\";\
#TFile *input = new TFile(\"/home/yoren/bnl/PHENIX/ee/AuAu/my-10M_phi_embed_helios_v1.root\", \"READ\");\
#auto ev = MyDileptonAnalysis::MyEvent();\
#TTree *T = (TTree *)input->Get(\"tree\");\
#TBranch *br = T->GetBranch(\"MyEvent\");\
#br->SetAddress(&myevent);"
#); 

In [13]:
from ROOT import ev, br

ImportError: cannot import name 'ev' from 'ROOT' (/home/yoren/bnl/ROOT/install/lib/ROOT/__init__.py)

In [25]:
print(ev)
print(ev.GetCentrality())

Name: MyDileptonAnalysis::MyEvent Title: MyEvent structure
-999.0


In [9]:
type_loc = 0
psi_names = ["BBC","FVTX"]
N_type_loc= len(hists_all[type_loc])
print(N_type_loc)
c0 = root.TCanvas(f"c0",f"c0",1400,250*N_centr)
c0.Divide(N_type_loc,N_centr)
legends=[]
for itype in range(N_type_loc):
    for icentr in range(N_centr):
        c0.cd(itype+icentr*N_type_loc+1)
        #ibin0 = hists_read[type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+0])
        #ibin1 = hists_read[type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+1])
        #hists_read[iFile][type_loc][itype].GetZaxis().SetRange(ibin0,ibin1)
        #hists_read[type_loc][itype].Project3D("yx").Draw("colz")
        proj = hists_read[type_loc][itype].ProjectionX(f"psi_{itype}_{icentr}",int(central_bins[2*icentr]/5)+1,int(central_bins[2*icentr+1]/5),3,3)
        Format_Hist_total(proj,title_x="#psi, rad",  title_y="counts", left=0.15, bottom=0.15, right=0, top=0,  Tsize=0.07,  Lsize=0.06,\
                      Mstyle=21,  Msize=2, Mcolor=4,  Lwidth=3,  Lcolor=4,  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
        proj.Draw()
        legends.append(Draw_Legend_Title(0.2,0.1,0.8,0.7,f"{psi_names[itype]}, {central_bins[2*icentr]}-{central_bins[2*icentr+1]}%",TextSize=0.08, align =22, LineWidth=0, Color=1,  alpha=0))
        legends[-1].Draw()

c0.Draw()
c0.SaveAs(f"output/qa_pt_centr.png")

2


In [10]:
type_loc = 1
psi_names = ["BBC","FVTX"]
N_type_loc= len(hists_all[type_loc])
print(N_type_loc)
c0 = root.TCanvas(f"c0",f"c0",1400,500)
c0.Divide(N_type_loc,1)
legends=[]
graphs = []
h = []
resoluton = []
for itype in range(N_type_loc):
    resoluton_type = []
    graphs.append(root.TGraphErrors(N_centr))
    c0.cd(itype+1)
    h.append(Format_Pad_old( left=0,  right=100,  min_boarder=0,  max_boarder=0.9,  title_x="centrality",  title_y="RES{#Psi}",  \
                           offset_x=1,  offset_y=1,  Tsize=0.05,  Lsize=0.05, title="", \
                           lefty=0.15, bottom=0.15, righty=0, top=0, name = "sec"))
    for icentr in range(N_centr):
        #ibin0 = hists_read[type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+0])
        #ibin1 = hists_read[type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+1])
        #hists_read[iFile][type_loc][itype].GetZaxis().SetRange(ibin0,ibin1)
        #hists_read[type_loc][itype].Project3D("yx").Draw("colz")
        proj0 = hists_read[type_loc][itype].ProjectionX(f"psi0_{itype}_{icentr}",int(central_bins[2*icentr]/5)+1,int(central_bins[2*icentr+1]/5),1,1)
        proj1 = hists_read[type_loc][itype].ProjectionX(f"psi1_{itype}_{icentr}",int(central_bins[2*icentr]/5)+1,int(central_bins[2*icentr+1]/5),2,2)
        proj2 = hists_read[type_loc][itype].ProjectionX(f"psi2_{itype}_{icentr}",int(central_bins[2*icentr]/5)+1,int(central_bins[2*icentr+1]/5),3,3)
        mean0, mean1, mean2 = proj0.GetMean(), proj1.GetMean(), proj2.GetMean()
        mean_err0, mean_err1, mean_err2 = proj0.GetMeanError(), proj1.GetMeanError(), proj2.GetMeanError()
        x = (central_bins[2*icentr]+central_bins[2*icentr+1])/2
        x_err = 2
        res = ( mean0 * mean1 / mean2 )**0.5
        resoluton_type.append(res)
        res_err = res * ( (mean_err0/mean0)**2 + (mean_err1/mean1)**2 + (mean_err2/mean2)**2  )**0.5
        graphs[-1].SetPoint(icentr,x, res)
        graphs[-1].SetPointError(icentr,x_err,res_err)
        print(itype,icentr,mean0, mean1, mean2)
    resoluton.append(resoluton_type)
    Format_Graph( graphs[-1],  Mstyle=20+itype,  Msize=1.4,  Mcolor=1+itype,  Lwidth=1,  Lcolor=1+itype,  Malpha=1,  Lalpha=1)
    graphs[-1].Draw("same P")
    legends.append(Draw_Legend_Title(0.6,0.5,0.8,0.9,f"{psi_names[itype]}",TextSize=0.08, align =22, LineWidth=0, Color=1,  alpha=0))
    legends[-1].Draw()

c0.Draw()
c0.SaveAs(f"output/qa_pt_centr.png")

2
0 0 0.16519619496454505 0.16568297633989978 0.2444576554473919
0 1 0.16026883880963103 0.16033625332051243 0.23631880045408538
0 2 0.22998782887670202 0.2307764491778278 0.3354250763619672
0 3 0.13157391004093097 0.13280222291044425 0.20420003131272338
0 4 0.03079869485335896 0.03132469712229554 0.051711366970443
1 0 0.14948320028288611 0.1498621148945975 0.054140554681268674
1 1 0.14524095767431688 0.1458429542799468 0.05324704193373874
1 2 0.2058053685159571 0.20634134447326014 0.07755729298443932
1 3 0.12177019950542965 0.12167809188253134 0.03818102223259663
1 4 0.03023725627567708 0.0296870970412442 0.006700505252317566


In [11]:
type_loc = 2
psi_names = ["BBC","FVTX"]
N_type_loc= len(hists_all[type_loc])
print(N_type_loc)
c0 = root.TCanvas(f"c2",f"c2",1400,400*N_centr)
c0.Divide(N_type_loc,N_centr)
legends=[]
h = []
myexps = []
for itype in range(N_type_loc):
    myexps_type = []
    for icentr in range(N_centr):
        c0.cd(itype+icentr*N_type_loc+1)
        graphs.append(root.TGraphErrors(N_centr))
        #ibin0 = hists_read[type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+0])
        #ibin1 = hists_read[type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+1])
        #hists_read[iFile][type_loc][itype].GetZaxis().SetRange(ibin0,ibin1)
        #hists_read[type_loc][itype].Project3D("yx").Draw("colz")
        proj = hists_read[type_loc][itype].ProjectionY(f"pt_{itype}_{icentr}",1,200,int(central_bins[2*icentr]/5)+1,int(central_bins[2*icentr+1]/5))
        Format_Hist_total(proj,title_x="p_{T}, GeV",  title_y="counts", left=0.15, bottom=0.15, right=0, top=0,  Tsize=0.07,  Lsize=0.06,\
                      Mstyle=21,  Msize=2, Mcolor=4,  Lwidth=3,  Lcolor=4,  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
        proj.Draw()
        myexps_type.append(root.TF1(f"myexp_{itype}_{icentr}","([0]*0.1*([1]-1)*([1]-2))*(pow((sqrt(x*x+[3]*[3])+[2]-[3])/([2]),-[1]))/(2*3.14159)/([2]+[3]*([1]-2))/([2]+[3])",0.2,5))
        myexps_type[-1].SetParameter(3,0.135)
        myexps_type[-1].SetParameter(0,proj.GetMaximum()*12)
        myexps_type[-1].SetParameter(1,15)
        myexps_type[-1].SetParameter(2,4.7)
        proj.Fit(myexps_type[-1],"Q","",1.3,5)
  
        root.gPad.SetLogy()
        legends.append(Draw_Legend_Title(0.2,0.1,0.8,0.7,f"{psi_names[itype]}, {central_bins[2*icentr]}-{central_bins[2*icentr+1]}%",TextSize=0.08, align =22, LineWidth=0, Color=1,  alpha=0))
        legends[-1].Draw()
    myexps.append(myexps_type)

c0.Draw()
c0.SaveAs(f"output/qa_pt_centr.png")

2


In [12]:
pt = []
pt_err = []
N_pt = len(pt_boarders) - 1
N_steps = 100
for itype in range(len(hists_all[2])):
    pt_itype = []
    pt_err_itype = []
    for icentr in range(N_centr):
        pt_icentr = []
        pt_err_icentr = []
        for i in range(N_pt):
            pt_step = pt_boarders[i]+(pt_boarders[i+1]-pt_boarders[i])/N_steps*(0+0.5)
            for istep in range(N_steps):
                pt_step = pt_boarders[i]+(pt_boarders[i+1]-pt_boarders[i])/N_steps*(istep+0.5)
                if myexps[itype][icentr].Integral(pt_boarders[i],pt_step,1e-15) > myexps[itype][icentr].Integral(pt_step,pt_boarders[i+1],1e-15):
                    break
            pt_icentr.append( round(pt_step,4))
            pt_err_icentr.append((pt_boarders[i+1]-pt_boarders[i])/100*2)
        pt_itype.append(pt_icentr)
        pt_err_itype.append(pt_err_icentr)
    pt.append(pt_itype)
    pt_err.append(pt_err_itype)
print("pt = ", pt,"\npt_boarders = ",pt_boarders)

pt =  [[[0.2415, 0.3415, 0.4425, 0.5435, 0.6435, 0.777, 0.979, 1.2095, 1.5575, 2.089, 2.775, 3.8675], [0.2415, 0.3415, 0.4425, 0.5425, 0.6435, 0.775, 0.979, 1.2065, 1.5525, 2.089, 2.765, 3.8675], [0.2425, 0.3425, 0.4435, 0.5435, 0.6435, 0.777, 0.979, 1.2095, 1.5575, 2.095, 2.775, 3.8825], [0.2425, 0.3435, 0.4435, 0.5435, 0.6445, 0.779, 0.981, 1.2125, 1.5625, 2.095, 2.775, 3.8825], [0.2405, 0.3405, 0.4415, 0.5425, 0.6425, 0.775, 0.977, 1.2065, 1.5525, 2.083, 2.765, 3.8675]], [[0.2415, 0.3415, 0.4425, 0.5435, 0.6435, 0.777, 0.979, 1.2095, 1.5575, 2.089, 2.775, 3.8675], [0.2415, 0.3415, 0.4425, 0.5425, 0.6435, 0.775, 0.979, 1.2065, 1.5525, 2.089, 2.765, 3.8675], [0.2425, 0.3425, 0.4435, 0.5435, 0.6435, 0.777, 0.979, 1.2095, 1.5575, 2.095, 2.775, 3.8825], [0.2425, 0.3435, 0.4435, 0.5435, 0.6445, 0.779, 0.981, 1.2125, 1.5625, 2.095, 2.775, 3.8825], [0.2405, 0.3405, 0.4415, 0.5425, 0.6425, 0.775, 0.977, 1.2065, 1.5525, 2.083, 2.765, 3.8675]]] 
pt_boarders =  [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.

In [13]:
type_loc = 2
psi_names = ["BBC","FVTX"]
N_type_loc= len(hists_all[type_loc])
print(N_type_loc)
c0 = root.TCanvas(f"c0",f"c0",1400,250*N_centr)
c0.Divide(N_type_loc,3)
legends=[]
graphs = []
h = []
for icentr in range(N_centr):
    c0.cd(icentr+1)
    h.append(Format_Pad_old( left=0,  right=5,  min_boarder=0,  max_boarder=0.4,  title_x="p_{T} [GeV/c]",  title_y="v_{2}",  \
                           offset_x=1,  offset_y=1,  Tsize=0.05,  Lsize=0.05, title="", \
                           lefty=0.15, bottom=0.15, righty=0, top=0, name = "sec"))
    legends.append(Draw_Legend_Title(0.25,0.6,0.75,0.8,f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%",TextSize=0.08, align =22, LineWidth=0, Color=1,  alpha=0))
    legends[-1].SetNColumns(3)
    for itype in range(N_type_loc):
        for ipt in range(N_pt):
            graphs.append(root.TGraphErrors(N_pt))
            v2_proj = hists_read[type_loc][itype].ProjectionX(f"v2_{itype}_{icentr}_{ipt}",int(pt_boarders[ipt]*10)+1,int(pt_boarders[ipt+1]*10)\
                                                            ,int(central_bins[2*icentr]/5)+1,int(central_bins[2*icentr+1]/5))
            v2 = v2_proj.GetMean()/resoluton[itype][icentr]
            v2_err = v2_proj.GetMeanError()/resoluton[itype][icentr]
            x_pt = pt[itype][icentr][ipt]
            x_pt_err = pt_err[itype][icentr][ipt]
            graphs[-1].SetPoint(icentr,x_pt, v2)
            graphs[-1].SetPointError(icentr,x_pt_err,v2_err)
            Format_Graph( graphs[-1],  Mstyle=20+itype,  Msize=1.2,  Mcolor=1+itype,  Lwidth=1,  Lcolor=1+itype,  Malpha=1,  Lalpha=1)
            graphs[-1].Draw("same P")
        legends[-1].AddEntry(graphs[-1],psi_names[itype],"lep")
    legends[-1].Draw()
    
c0.Draw()
c0.SaveAs(f"output/qa_pt_centr.png")

2


In [14]:
type_loc = 2
psi_choice = 1
part_names = ["e^{+}","e^{-}"]
N_type_loc= len(hists_all[type_loc])
print(N_type_loc)
c0 = root.TCanvas(f"c0",f"c0",1400,250*N_centr)
c0.Divide(N_type_loc,3)
legends=[]
graphs = []
h = []
for icentr in range(N_centr):
    c0.cd(icentr+1)
    h.append(Format_Pad_old( left=0,  right=5,  min_boarder=0,  max_boarder=0.4,  title_x="p_{T} [GeV/c]",  title_y="v_{2}",  \
                           offset_x=1,  offset_y=1,  Tsize=0.05,  Lsize=0.05, title="", \
                           lefty=0.15, bottom=0.15, righty=0, top=0, name = "sec"))
    legends.append(Draw_Legend_Title(0.2,0.6,0.9,0.8,f"FVTX {central_bins[2*icentr]}-{central_bins[2*icentr+1]}%",TextSize=0.08, align =22, LineWidth=0, Color=1,  alpha=0))
    legends[-1].SetNColumns(3)
    for ipart in range(2):
        for ipt in range(N_pt):
            graphs.append(root.TGraphErrors(N_pt))
            v2_proj = hists_read[type_loc][psi_choice].ProjectionX(f"v2_charge_{ipart}_{icentr}_{ipt}",int(pt_boarders[ipt]*10)+1,int(pt_boarders[ipt+1]*10)\
                                                            ,int(central_bins[2*icentr]/5+20*ipart)+1,int(central_bins[2*icentr+1]/5+20*ipart))
            v2 = v2_proj.GetMean()/resoluton[psi_choice][icentr]
            v2_err = v2_proj.GetMeanError()/resoluton[psi_choice][icentr]
            x_pt = pt[psi_choice][icentr][ipt]
            x_pt_err = pt_err[psi_choice][icentr][ipt]
            graphs[-1].SetPoint(icentr,x_pt, v2)
            graphs[-1].SetPointError(icentr,x_pt_err,v2_err)
            Format_Graph( graphs[-1],  Mstyle=20+ipart,  Msize=1.2,  Mcolor=1+ipart,  Lwidth=1,  Lcolor=1+ipart,  Malpha=1,  Lalpha=1)
            graphs[-1].Draw("same P")
        legends[-1].AddEntry(graphs[-1],part_names[ipart],"lep")
    legends[-1].Draw()
    
c0.Draw()
c0.SaveAs(f"output/qa_pt_centr.png")

2
